In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Importing Dataset

In [2]:
data_url = (
    "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
)
raw_data = pd.read_csv(data_url, header=None)
print(f"Dataset shape: {raw_data.shape}")
raw_data.head()

Dataset shape: (581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


Convert the one hot encoded input to label encoding. And setting column headers.

In [10]:
soil_type_values = [f"soil_type_{idx+1}" for idx in range(40)]
wilderness_area_values = [f"area_type_{idx+1}" for idx in range(4)]

soil_type = raw_data.loc[:, 14:53].apply(
    lambda x: soil_type_values[0::1][x.to_numpy().nonzero()[0][0]], axis=1
)
wilderness_area = raw_data.loc[:, 10:13].apply(
    lambda x: wilderness_area_values[0::1][x.to_numpy().nonzero()[0][0]], axis=1
)

CSV_HEADER = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
    "Wilderness_Area",
    "Soil_Type",
    "Cover_Type",
]

data = pd.concat(
    [raw_data.loc[:, 0:9], wilderness_area, soil_type, raw_data.loc[:, 54]],
    axis=1,
    ignore_index=True,
)
data.columns = CSV_HEADER

# Convert the target label indices into a range from 0 to 6 (there are 7 labels in total).
data["Cover_Type"] = data["Cover_Type"] - 1

print(f"Dataset shape: {data.shape}")
data.head()

Dataset shape: (581012, 13)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,area_type_1,soil_type_29,4
1,2590,56,2,212,-6,390,220,235,151,6225,area_type_1,soil_type_29,4
2,2804,139,9,268,65,3180,234,238,135,6121,area_type_1,soil_type_12,1
3,2785,155,18,242,118,3090,238,238,122,6211,area_type_1,soil_type_30,1
4,2595,45,2,153,-1,391,220,234,150,6172,area_type_1,soil_type_29,4


#### Train-Test Split
Split the data into training (85%) and validation (15%) sets.

In [4]:
train_splits = []
test_splits = []

for _, group_data in data.groupby("Cover_Type"):
    random_selection = np.random.rand(len(group_data.index)) <= 0.85
    train_splits.append(group_data[random_selection])
    test_splits.append(group_data[~random_selection])

train_data = pd.concat(train_splits).sample(frac=1).reset_index(drop=True)
test_data = pd.concat(test_splits).sample(frac=1).reset_index(drop=True)

print(f"Train split size: {len(train_data.index)}")
print(f"Test split size: {len(test_data.index)}")

Train split size: 493789
Test split size: 87223


Define variables that'll help in reading and parsing the data.

In [5]:
TARGET_FEATURE_NAME = "Cover_Type"

TARGET_FEATURE_LABELS = ["0", "1", "2", "3", "4", "5", "6"]

NUMERIC_FEATURE_NAMES = [
    "Aspect",
    "Elevation",
    "Hillshade_3pm",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Horizontal_Distance_To_Fire_Points",
    "Horizontal_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Slope",
    "Vertical_Distance_To_Hydrology",
]

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "Soil_Type": list(data["Soil_Type"].unique()),
    "Wilderness_Area": list(data["Wilderness_Area"].unique()),
}

CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

COLUMN_DEFAULTS = [
    [0] if feature_name in NUMERIC_FEATURE_NAMES + [TARGET_FEATURE_NAME] else ["NA"]
    for feature_name in CSV_HEADER
]

NUM_CLASSES = len(TARGET_FEATURE_LABELS)

Making test and train csv files.

In [6]:
train_data_file = "train_data.csv"
test_data_file = "test_data.csv"

train_data.to_csv(train_data_file, index=False)
test_data.to_csv(test_data_file, index=False)

An input function that reads file, then converts features and labels into tf.data.Dataset for training or evaluation.

In [7]:
def get_dataset_from_csv(csv_file_path, batch_size, shuffle=False):

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=True,
        shuffle=shuffle,
    )
    return dataset.cache()

Setting model parameters.

In [8]:
learning_rate = 0.001
dropout_rate = 0.1
batch_size = 265
num_epochs = 50

hidden_units = [32, 32]


def run_experiment(model):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)

    test_dataset = get_dataset_from_csv(test_data_file, batch_size)

    print("Start training the model...")
    history = model.fit(train_dataset, epochs=num_epochs)
    print("Model training finished")

    _, accuracy = model.evaluate(test_dataset, verbose=0)

    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Creating inputs, setting separate Input Layers for Categorical and Numerical values.

In [9]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs

Create two representations of the input features: sparse and dense:
1. In the sparse representation, the categorical features are encoded with one-hot
encoding using the CategoryEncoding layer. This representation can be useful for the
model to memorize particular feature values to make certain predictions.
2. In the dense representation, the categorical features are encoded with low-dimensional
embeddings using the Embedding layer. This representation helps the model
to generalize well to unseen feature combinations.

In [12]:
from tensorflow.keras.layers import StringLookup


def encode_inputs(inputs, use_embedding=False):
    encoded_features = []
    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int" if use_embedding else "binary",
            )
            if use_embedding:
                # Convert the string input values into integer indices.
                encoded_feature = lookup(inputs[feature_name])
                embedding_dims = int(math.sqrt(len(vocabulary)))
                # Create an embedding layer with the specified dimensions.
                embedding = layers.Embedding(
                    input_dim=len(vocabulary), output_dim=embedding_dims
                )
                # Convert the index values to embedding representations.
                encoded_feature = embedding(encoded_feature)
            else:
                # Convert the string input values into a one hot encoding.
                encoded_feature = lookup(tf.expand_dims(inputs[feature_name], -1))
        else:
            # Use the numerical features as-is.
            encoded_feature = tf.expand_dims(inputs[feature_name], -1)

        encoded_features.append(encoded_feature)

    all_features = layers.concatenate(encoded_features)
    return all_features

#### Model 1 :
Create a multi-layer feed-forward network, where the categorical features are one-hot
encoded.
learning_rate = 0.001
dropout_rate = 0.1
batch_size = 265
num_epochs = 50
hidden_units = [32, 32]

In [13]:
def create_baseline_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)

    for units in hidden_units:
        features = layers.Dense(units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.ReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model()

/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [14]:
run_experiment(baseline_model)

Start training the model...
Epoch 1/50
1864/1864 [==============================] - 12s 6ms/step - loss: 0.7650 - sparse_categorical_accuracy: 0.6821
Epoch 2/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.6663 - sparse_categorical_accuracy: 0.7139
Epoch 3/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.6441 - sparse_categorical_accuracy: 0.7220
Epoch 4/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.6260 - sparse_categorical_accuracy: 0.7313
Epoch 5/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.6132 - sparse_categorical_accuracy: 0.7368
Epoch 6/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.6024 - sparse_categorical_accuracy: 0.7404
Epoch 7/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.5942 - sparse_categorical_accuracy: 0.7441
Epoch 8/50
1864/1864 [==============================] - 5s 3ms/step - loss: 0.5879 - sparse_categorical_accuracy: 0.7464
Epo

This simple model achieves 74% accuracy.

### Wide & Deep Model
The wide part of the model a linear model, while the deep part of the model is a multi-layer feed-forward network.

Use the sparse representation of the input features in the wide part of the model and the dense representation of the input features for the deep part of the model.

Note that every input features contributes to both parts of the model with different representations.

In [16]:
def create_wide_and_deep_model():
  # Get inputs
    inputs = create_model_inputs()
    # Get sparse representation
    wide = encode_inputs(inputs)
    wide = layers.BatchNormalization()(wide)
#Get dense representation
    deep = encode_inputs(inputs, use_embedding=True)
    #Make model architecture for deep part
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)
# merge the final outputs of the dense and sparse 
    merged = layers.concatenate([wide, deep])
    # send them to the final softmax layer
    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


wide_and_deep_model = create_wide_and_deep_model()

In [17]:
run_experiment(wide_and_deep_model)

Start training the model...
Epoch 1/50
1864/1864 [==============================] - 13s 6ms/step - loss: 0.7027 - sparse_categorical_accuracy: 0.7067
Epoch 2/50
1864/1864 [==============================] - 7s 4ms/step - loss: 0.6036 - sparse_categorical_accuracy: 0.7379
Epoch 3/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5863 - sparse_categorical_accuracy: 0.7452
Epoch 4/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5742 - sparse_categorical_accuracy: 0.7510
Epoch 5/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5658 - sparse_categorical_accuracy: 0.7554
Epoch 6/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5604 - sparse_categorical_accuracy: 0.7581
Epoch 7/50
1864/1864 [==============================] - 7s 4ms/step - loss: 0.5531 - sparse_categorical_accuracy: 0.7611
Epoch 8/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5484 - sparse_categorical_accuracy: 0.7634
Epo

Wide and deep gets us about 80% test accuracy.

### Deep & Cross model
In the third experiment, we create a Deep & Cross model. The deep part of this model is the same as the deep part created in the previous experiment. The key idea of the cross part is to apply explicit feature crossing in an efficient way, where the degree of cross features grows with layer depth.

In [20]:
def create_deep_and_cross_model():
    inputs = create_model_inputs()
    x0 = encode_inputs(inputs, use_embedding=True)
    cross = x0
    for _ in hidden_units:
        units = cross.shape[-1]
        x = layers.Dense(units)(cross)
        cross = x0 * x + cross
    cross = layers.BatchNormalization()(cross)
    
    deep = x0
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    merged = layers.concatenate([cross, deep])
    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


deep_and_cross_model = create_deep_and_cross_model()

In [21]:
run_experiment(deep_and_cross_model)

Start training the model...
Epoch 1/50
1864/1864 [==============================] - 12s 6ms/step - loss: 0.6914 - sparse_categorical_accuracy: 0.7086
Epoch 2/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5914 - sparse_categorical_accuracy: 0.7433
Epoch 3/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5733 - sparse_categorical_accuracy: 0.7506
Epoch 4/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5612 - sparse_categorical_accuracy: 0.7556
Epoch 5/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5535 - sparse_categorical_accuracy: 0.7601
Epoch 6/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5468 - sparse_categorical_accuracy: 0.7636
Epoch 7/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5418 - sparse_categorical_accuracy: 0.7657
Epoch 8/50
1864/1864 [==============================] - 6s 3ms/step - loss: 0.5379 - sparse_categorical_accuracy: 0.7677
Epo

The Test Accuracy is 80% again.